# Taller Práctico: Introducción a LangChain y Agentes Inteligentes

## Contexto del Taller

En este taller aprenderás a construir aplicaciones inteligentes utilizando LangChain, una librería que facilita el desarrollo de sistemas basados en modelos de lenguaje. Trabajaremos con un caso práctico: análisis automatizado de reseñas de restaurantes.

## Objetivos Educativos

- Comprender los conceptos fundamentales de LangChain y su arquitectura
- Implementar prompts estructurados usando PromptTemplate
- Conectar y utilizar modelos de lenguaje (Gemini) en aplicaciones reales
- Procesar y parsear las respuestas de los modelos de forma estructurada
- Trabajar con documentos externos usando Document Loaders
- Crear agentes inteligentes capaces de usar herramientas
- Serializar y persistir configuraciones de prompts

## Resultados Esperados

Al finalizar este taller, serás capaz de:
- Configurar un entorno de trabajo con LangChain en Google Colab
- Diseñar prompts efectivos para tareas específicas
- Construir pipelines de procesamiento de texto con LLMs
- Implementar agentes básicos que tomen decisiones y usen herramientas
- Aplicar estos conocimientos a casos de uso reales

## Estructura del Taller

1. Configuración del entorno
2. Primeros pasos con LangChain y Gemini
3. PromptTemplates: Diseño de instrucciones efectivas
4. Parsing y procesamiento de salidas
5. Serialización de prompts
6. Carga y procesamiento de documentos
7. Resumen automático de documentos
8. Creación de agentes con herramientas

---

## 1. Configuración del Entorno

Antes de comenzar, necesitamos instalar las librerías necesarias y configurar el acceso a la API de Gemini. Google Colab nos proporciona acceso gratuito a la API de Gemini, lo cual es ideal para aprender.

### ¿Qué vamos a instalar?

- **langchain**: Framework principal para trabajar con LLMs
- **langchain-google-genai**: Integración específica para modelos de Google
- **pypdf**: Para leer archivos PDF
- **faiss-cpu**: Base de datos vectorial para búsquedas semánticas

In [15]:
# Instalación de dependencias
# Esto puede tomar un par de minutos la primera vez
!pip install -q langchain langchain-google-genai langchain-community pypdf faiss-cpu

print("Instalación completada correctamente")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.
Instalación completada correctamente


### Configuración de la API Key

Google Colab nos proporciona acceso gratuito a Gemini. Para obtener tu API key:
1. Ve a https://aistudio.google.com/app/apikey
2. Inicia sesión con tu cuenta de Google
3. Crea una nueva API key
4. Cópiala y pégala cuando se te solicite

**Nota importante sobre seguridad**: Usar `userdata` en Colab es la forma segura de manejar API keys. Nunca incluyas tu API key directamente en el código.

In [2]:
# Importamos las librerías necesarias
import os
from google.colab import userdata

# Configuramos la API key de forma segura
# En Colab: Click en la llave (🔑) en el panel izquierdo y agrega tu API key con el nombre "GOOGLE_API_KEY"
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

print("Configuración completada correctamente")

Configuración completada correctamente


---

## 2. Primer Contacto con LangChain y Gemini

Ahora que tenemos todo configurado, vamos a realizar nuestra primera interacción con un modelo de lenguaje. LangChain actúa como un puente entre nuestra aplicación y el modelo, simplificando la comunicación.

### Conceptos clave:
- **LLM (Large Language Model)**: El modelo de inteligencia artificial que procesa el texto
- **ChatGoogleGenerativeAI**: La clase de LangChain que conecta con Gemini
- **Invoke**: El método que envía el prompt al modelo y recibe la respuesta

In [3]:
# Importamos el modelo de chat de Google
from langchain_google_genai import ChatGoogleGenerativeAI

# Inicializamos el modelo Gemini
# Usamos gemini-2.5-flash porque es rápido y gratuito
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7  # Controla la creatividad: 0 = más determinista, 1 = más creativo
)

# Realizamos nuestra primera consulta
respuesta = llm.invoke("Explica en una oración qué es LangChain")

print("Respuesta del modelo:")
print(respuesta.content)

Respuesta del modelo:
LangChain es un framework que facilita la creación de aplicaciones con modelos de lenguaje grandes (LLMs) al permitirles interactuar con datos externos y otras herramientas.


### Reflexión

Acabas de interactuar con un modelo de lenguaje a través de LangChain. Aunque parece simple, este patrón es la base de aplicaciones más complejas.

**Pregunta para reflexionar**: ¿Qué pasaría si cambias el valor de `temperature` a 0.1 o a 0.9? Pruébalo y observa las diferencias.

---

## 3. System Prompts: Definiendo el Comportamiento del Modelo

Hasta ahora hemos enviado consultas directas al modelo. Sin embargo, en aplicaciones reales necesitamos que el modelo tenga un comportamiento consistente y específico. Aquí es donde entran los **System Prompts**.

### ¿Qué es un System Prompt?

El System Prompt es una instrucción especial que define:
- **El rol** que debe asumir el modelo (ej: experto en gastronomía, asistente técnico, etc.)
- **El tono** de las respuestas (formal, casual, técnico, educativo)
- **Las restricciones** o reglas que debe seguir
- **El contexto** en el que opera

A diferencia de los mensajes del usuario, el System Prompt se mantiene constante durante toda la conversación y el modelo lo prioriza al generar respuestas.

### Diferencia entre System Prompt y User Prompt

- **System Prompt**: "Eres un crítico gastronómico profesional con 20 años de experiencia"
- **User Prompt**: "Analiza esta reseña de restaurante"

El System Prompt establece el marco de trabajo, mientras que el User Prompt contiene la tarea específica.

### Ejemplo 1: Sin System Prompt vs Con System Prompt

Vamos a comparar las respuestas del modelo con y sin System Prompt usando nuestro caso de análisis de reseñas.

In [4]:
# Importamos las clases necesarias para mensajes estructurados
from langchain_core.messages import HumanMessage, SystemMessage

# Reseña de prueba
resena_ejemplo = """
La pizza estaba buena pero el servicio fue lento.
Tuvimos que esperar 30 minutos. Los precios son razonables.
"""

print("=" * 70)
print("PRUEBA 1: SIN SYSTEM PROMPT")
print("=" * 70)

# Sin System Prompt - solo el mensaje del usuario
respuesta_sin_system = llm.invoke([
    HumanMessage(content=f"Analiza esta reseña de restaurante: {resena_ejemplo}")
])

print(respuesta_sin_system.content)

print("\n" + "=" * 70)
print("PRUEBA 2: CON SYSTEM PROMPT PROFESIONAL")
print("=" * 70)

# Con System Prompt - definimos el rol del modelo
respuesta_con_system = llm.invoke([
    SystemMessage(content="""Eres un analista profesional de reseñas de restaurantes.
    Tu trabajo es analizar reseñas de forma objetiva y estructurada.
    Siempre debes incluir:
    - Sentimiento general (Positivo/Negativo/Neutral)
    - Aspectos específicos evaluados
    - Calificación numérica del 1 al 5
    - Mantén un tono profesional y conciso"""),
    HumanMessage(content=f"Analiza esta reseña: {resena_ejemplo}")
])

print(respuesta_con_system.content)

PRUEBA 1: SIN SYSTEM PROMPT
Esta reseña, aunque breve, proporciona información valiosa y concisa sobre la experiencia del cliente. Aquí tienes un análisis detallado:

**Análisis de la Reseña del Restaurante**

La reseña se puede dividir en tres categorías principales: calidad del producto, calidad del servicio y relación calidad-precio.

**1. Calidad del Producto (Positivo):**
*   **"La pizza estaba buena"**: Este es el punto más fuerte y central de la reseña. Indica que el producto principal del establecimiento (la pizza) cumple con las expectativas y es de buena calidad. Esto es crucial para un restaurante especializado en pizza.

**2. Calidad del Servicio (Negativo):**
*   **"pero el servicio fue lento"**: Este es el principal inconveniente y se presenta como un contraste directo a la buena calidad de la comida. La lentitud en el servicio es una queja común y puede afectar significativamente la experiencia general del cliente.
*   **"Tuvimos que esperar 30 minutos."**: Esta es una c

### Ejemplo 2: Diferentes System Prompts para diferentes personalidades

El mismo input puede generar respuestas muy diferentes según el System Prompt. Veamos cómo cambiar el tono del análisis.

In [5]:
# System Prompt 1: Crítico gastronómico exigente
system_critico_exigente = """Eres un crítico gastronómico de alta cocina con estándares muy elevados.
Tienes años de experiencia evaluando restaurantes de lujo. Eres muy detallista y exigente.
Siempre señalas áreas de mejora y comparas con estándares internacionales."""

# System Prompt 2: Blogger casual y amigable
system_blogger_casual = """Eres un blogger de comida que escribe reseñas accesibles y amigables.
Tu audiencia son personas comunes buscando recomendaciones confiables.
Usas un lenguaje cercano y te enfocas en la experiencia general del cliente promedio."""

# System Prompt 3: Analista de datos objetivo
system_analista_datos = """Eres un analista de datos especializado en análisis de sentimientos.
Evalúas reseñas de forma puramente objetiva usando métricas cuantificables.
Presentas tus análisis de forma estructurada con categorías claras y puntuaciones numéricas."""

resena_test = "La comida estaba bien, nada del otro mundo. El servicio fue amable."

# Probamos con cada System Prompt
print("=" * 70)
print("CRÍTICO EXIGENTE")
print("=" * 70)
resp1 = llm.invoke([
    SystemMessage(content=system_critico_exigente),
    HumanMessage(content=f"Analiza esta reseña: {resena_test}")
])
print(resp1.content)

print("\n" + "=" * 70)
print("BLOGGER CASUAL")
print("=" * 70)
resp2 = llm.invoke([
    SystemMessage(content=system_blogger_casual),
    HumanMessage(content=f"Analiza esta reseña: {resena_test}")
])
print(resp2.content)

print("\n" + "=" * 70)
print("ANALISTA DE DATOS")
print("=" * 70)
resp3 = llm.invoke([
    SystemMessage(content=system_analista_datos),
    HumanMessage(content=f"Analiza esta reseña: {resena_test}")
])
print(resp3.content)

CRÍTICO EXIGENTE
Permítame ser brutalmente honesto, como es mi costumbre cuando se trata de evaluar la cúspide de la gastronomía. Esta "reseña" no es una reseña; es un epitafio. Es un laconismo que, en el contexto de la alta cocina, resulta más devastador que una crítica pormenorizada y mordaz.

Analicemos esta sentencia de muerte a la experiencia culinaria:

---

**Análisis de la "Reseña": "La comida estaba bien, nada del otro mundo. El servicio fue amable."**

**1. Sobre la "Comida": "La comida estaba bien, nada del otro mundo."**

*   **"Estaba bien":** Esta frase es la antítesis de lo que se busca en un establecimiento de alta cocina. "Bien" es lo que uno espera de un bistró de barrio con pretensiones modestas, no de un templo gastronómico que justifica precios exorbitantes y una reputación construida sobre la excelencia. En este segmento, "bien" significa un fracaso rotundo en la ejecución, la concepción y la ambición. Implica una ausencia de técnica sublime, de ingredientes de pr

### Integrando System Prompts con ChatPromptTemplate

En aplicaciones reales, usamos `ChatPromptTemplate` para combinar System Prompts con templates de usuario de forma estructurada. Esto es especialmente útil cuando necesitamos consistencia en toda la aplicación.

In [6]:
# Importamos ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate

# Creamos un template completo con System Prompt integrado
prompt_con_system = ChatPromptTemplate.from_messages([
    ("system", """Eres un asistente especializado en análisis de reseñas de restaurantes.

    Tu responsabilidad es procesar reseñas y generar análisis estructurados que ayuden a:
    1. Propietarios de restaurantes a identificar áreas de mejora
    2. Clientes potenciales a tomar decisiones informadas

    REGLAS IMPORTANTES:
    - Siempre sé objetivo y basado en los hechos mencionados
    - Identifica aspectos específicos: comida, servicio, ambiente, precio
    - Asigna una calificación del 1 al 5
    - Si la información es insuficiente, indícalo claramente
    - Mantén un tono profesional pero accesible"""),

    ("human", """Restaurante: {restaurante}

Reseña del cliente:
{resena}

Por favor, genera un análisis estructurado.""")
])

# Creamos la cadena completa con System Prompt
cadena_con_system = prompt_con_system | llm

# Probamos con diferentes reseñas
resultado1 = cadena_con_system.invoke({
    "restaurante": "Bistró del Puerto",
    "resena": "Excelente ubicación frente al mar. El ceviche estaba fresco y delicioso. El mesero fue muy atento. Un poco caro pero vale la pena."
})

print("Análisis con System Prompt integrado:")
print(resultado1.content)

print("\n" + "=" * 70)

# Probamos con una reseña negativa
resultado2 = cadena_con_system.invoke({
    "restaurante": "Cafetería Central",
    "resena": "El café estaba frío y el pan duro. Tardaron mucho en atendernos."
})

print("\nAnálisis de reseña negativa:")
print(resultado2.content)

Análisis con System Prompt integrado:
**Análisis de Reseña: Bistró del Puerto**

**Aspectos Evaluados:**

*   **Comida:**
    *   **Detalles:** El cliente destaca que el ceviche estaba "fresco y delicioso".
    *   **Calificación:** 5/5
*   **Servicio:**
    *   **Detalles:** El mesero fue descrito como "muy atento".
    *   **Calificación:** 5/5
*   **Ambiente:**
    *   **Detalles:** Se menciona una "excelente ubicación frente al mar".
    *   **Calificación:** 5/5
*   **Precio:**
    *   **Detalles:** El cliente lo considera "un poco caro", pero aclara que "vale la pena".
    *   **Calificación:** 3/5 (Indica que es un factor a considerar, aunque el valor percibido es alto).

**Resumen General:**
La reseña es altamente positiva, destacando la calidad de la comida (específicamente el ceviche), la atención del servicio y la inmejorable ubicación del restaurante. El único punto de consideración es el precio, que el cliente percibe como elevado, aunque justifica el costo por la experien

### Buenas prácticas para System Prompts

Al diseñar System Prompts efectivos, considera:

**1. Claridad en el rol**
- Define específicamente quién es el modelo
- Establece el nivel de expertise esperado

**2. Instrucciones explícitas**
- Lista reglas claras sobre qué hacer y qué evitar
- Especifica el formato de salida esperado

**3. Contexto relevante**
- Proporciona información sobre el propósito de la tarea
- Incluye restricciones importantes

**4. Consistencia**
- Usa el mismo System Prompt para tareas similares
- Versiona tus System Prompts cuando los modifiques

**5. Testing**
- Prueba el System Prompt con casos extremos
- Valida que funcione con diferentes tipos de input

### Reflexión práctica

**Ejercicio**: Modifica el System Prompt anterior para que:
1. El análisis incluya recomendaciones de mejora para el restaurante
2. Clasifique el tipo de cliente que escribió la reseña (casual, experto, etc.)
3. Sugiera una respuesta apropiada del restaurante a la reseña

Intenta mantener las instrucciones claras y concisas.

---

## 4. PromptTemplates: Diseñando Instrucciones Efectivas

Los prompts son las instrucciones que le damos al modelo. Un `PromptTemplate` nos permite crear prompts reutilizables con variables que podemos cambiar dinámicamente. Esto es fundamental para construir aplicaciones escalables.

### Caso práctico: Análisis de reseñas de restaurantes

Imagina que trabajas para una plataforma de reseñas de restaurantes y necesitas clasificar automáticamente miles de comentarios. En lugar de escribir el prompt manualmente cada vez, creamos una plantilla.


In [7]:
# Importamos PromptTemplate
from langchain.prompts import PromptTemplate

# Creamos una plantilla para analizar reseñas
# Las variables van entre llaves {}
template_resena = """
Eres un analista de reseñas de restaurantes. Tu trabajo es analizar la siguiente reseña y proporcionar:
1. Sentimiento general (Positivo, Negativo o Neutral)
2. Aspectos destacados (comida, servicio, ambiente, precio)
3. Calificación estimada del 1 al 5

Reseña del cliente:
{resena}

Restaurante: {nombre_restaurante}

Proporciona un análisis claro y conciso.
"""

# Creamos el PromptTemplate
prompt_analisis = PromptTemplate(
    input_variables=["resena", "nombre_restaurante"],
    template=template_resena
)

# Probamos la plantilla con datos de ejemplo
resena_ejemplo = """
La pasta estaba deliciosa y el servicio fue muy atento.
Sin embargo, el tiempo de espera fue de casi 40 minutos y los precios son bastante elevados.
La decoración del lugar es hermosa, muy acogedor.
"""

# Formateamos el prompt con nuestros datos
prompt_formateado = prompt_analisis.format(
    resena=resena_ejemplo,
    nombre_restaurante="Trattoria Bella Vista"
)

print("Prompt generado:")
print(prompt_formateado)

Prompt generado:

Eres un analista de reseñas de restaurantes. Tu trabajo es analizar la siguiente reseña y proporcionar:
1. Sentimiento general (Positivo, Negativo o Neutral)
2. Aspectos destacados (comida, servicio, ambiente, precio)
3. Calificación estimada del 1 al 5

Reseña del cliente:

La pasta estaba deliciosa y el servicio fue muy atento. 
Sin embargo, el tiempo de espera fue de casi 40 minutos y los precios son bastante elevados.
La decoración del lugar es hermosa, muy acogedor.


Restaurante: Trattoria Bella Vista

Proporciona un análisis claro y conciso.



### Ejecutando el análisis con el modelo

Ahora conectamos nuestra plantilla con el modelo. LangChain nos permite crear "cadenas" (chains) que combinan prompts y modelos de forma elegante.

In [8]:
# Creamos una cadena: Prompt + Modelo
# El operador | (pipe) conecta el prompt con el modelo
cadena_analisis = prompt_analisis | llm

# Ejecutamos la cadena con nuestros datos
resultado = cadena_analisis.invoke({
    "resena": resena_ejemplo,
    "nombre_restaurante": "Trattoria Bella Vista"
})

print("Análisis generado por el modelo:")
print(resultado.content)

Análisis generado por el modelo:
Aquí está el análisis de la reseña para Trattoria Bella Vista:

---

**Análisis de Reseña - Trattoria Bella Vista**

1.  **Sentimiento general:** **Positivo**
    *   Aunque existen puntos negativos importantes, los aspectos positivos sobre la comida, el servicio y el ambiente son muy fuertes y dominantes, inclinando la balanza hacia una experiencia generalmente agradable.

2.  **Aspectos destacados:**
    *   **Comida:** Muy positiva (la pasta fue descrita como "deliciosa").
    *   **Servicio:** Muy positivo (descrito como "muy atento").
    *   **Ambiente:** Muy positivo (la decoración es "hermosa" y el lugar "muy acogedor").
    *   **Tiempo de Espera:** Negativo (casi 40 minutos, lo cual es considerable).
    *   **Precio:** Negativo (considerado "bastante elevados").

3.  **Calificación estimada del 1 al 5:** **4/5**
    *   La excelencia en comida, servicio y ambiente justifica una calificación alta. Sin embargo, el largo tiempo de espera y los p

### Ejercicio práctico

Prueba analizar esta otra reseña modificando los parámetros de la cadena:

"El servicio fue lento y la comida llegó fría. No lo recomendaría. El único punto positivo es que el lugar está bien ubicado."

Restaurante: "Café del Centro"

In [9]:
# Espacio para tu práctica
# Ejecuta el análisis con la nueva reseña

resultado_practica = cadena_analisis.invoke({
    "resena": "El servicio fue lento y la comida llegó fría. No lo recomendaría. El único punto positivo es que el lugar está bien ubicado.",
    "nombre_restaurante": "Café del Centro"
})

print(resultado_practica.content)

Aquí está el análisis de la reseña del cliente para Café del Centro:

---

**Análisis de la reseña del cliente - Café del Centro**

1.  **Sentimiento general:** Negativo
2.  **Aspectos destacados:**
    *   **Comida:** Negativo (llegó fría)
    *   **Servicio:** Negativo (lento)
    *   **Ambiente/Ubicación:** Positivo (bien ubicado)
    *   **Precio:** No mencionado
3.  **Calificación estimada:** 2 de 5


---

## 5. Parsing y Procesamiento de Salidas Estructuradas

Hasta ahora, las respuestas del modelo son texto libre. Pero para aplicaciones reales, necesitamos datos estructurados que podamos procesar programáticamente. LangChain ofrece "Output Parsers" para esto.

### ¿Por qué es importante?

Si estamos procesando miles de reseñas, necesitamos extraer datos específicos (como la calificación) de forma automática y confiable, no como texto narrativo.

In [10]:
# Importamos el parser estructurado
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# Definimos el esquema de la respuesta que esperamos
esquemas_respuesta = [
    ResponseSchema(name="sentimiento", description="El sentimiento general: Positivo, Negativo o Neutral"),
    ResponseSchema(name="calificacion", description="Calificación numérica del 1 al 5"),
    ResponseSchema(name="aspectos_positivos", description="Lista de aspectos positivos mencionados"),
    ResponseSchema(name="aspectos_negativos", description="Lista de aspectos negativos mencionados"),
    ResponseSchema(name="resumen", description="Resumen breve en una oración")
]

# Creamos el parser
parser_salida = StructuredOutputParser.from_response_schemas(esquemas_respuesta)

# Obtenemos las instrucciones de formato que debemos incluir en el prompt
instrucciones_formato = parser_salida.get_format_instructions()

print("Instrucciones de formato para el modelo:")
print(instrucciones_formato)

Instrucciones de formato para el modelo:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"sentimiento": string  // El sentimiento general: Positivo, Negativo o Neutral
	"calificacion": string  // Calificación numérica del 1 al 5
	"aspectos_positivos": string  // Lista de aspectos positivos mencionados
	"aspectos_negativos": string  // Lista de aspectos negativos mencionados
	"resumen": string  // Resumen breve en una oración
}
```


### Creando un prompt con salida estructurada

Ahora modificamos nuestro prompt para incluir las instrucciones de formato. El modelo aprenderá a responder en el formato JSON que necesitamos.

In [11]:
# Creamos un nuevo template que incluye las instrucciones de formato
template_estructurado = """
Analiza la siguiente reseña de restaurante y extrae la información solicitada.

Reseña: {resena}

{instrucciones_formato}
"""

# Creamos el nuevo prompt
prompt_estructurado = PromptTemplate(
    template=template_estructurado,
    input_variables=["resena"],
    partial_variables={"instrucciones_formato": instrucciones_formato}
)

# Creamos la cadena completa: Prompt -> Modelo -> Parser
cadena_estructurada = prompt_estructurado | llm | parser_salida

# Probamos con una reseña
resena_test = """
Excelente experiencia en este restaurante. La atención fue impecable y los meseros
muy amables. Los platos están bien presentados y el sabor es excepcional.
El único detalle es que el estacionamiento es limitado. Definitivamente volveré.
"""

# Ejecutamos y obtenemos la salida estructurada
resultado_estructurado = cadena_estructurada.invoke({"resena": resena_test})

print("Resultado estructurado (diccionario Python):")
print(resultado_estructurado)
print("\n--- Accediendo a campos específicos ---")
print(f"Sentimiento: {resultado_estructurado['sentimiento']}")
print(f"Calificación: {resultado_estructurado['calificacion']}")

Resultado estructurado (diccionario Python):
{'sentimiento': 'Positivo', 'calificacion': '5', 'aspectos_positivos': 'Excelente experiencia general, atención impecable, meseros muy amables, platos bien presentados, sabor excepcional.', 'aspectos_negativos': 'Estacionamiento limitado.', 'resumen': 'El restaurante ofrece una excelente experiencia con atención impecable, meseros amables, platos bien presentados y sabor excepcional, aunque el estacionamiento es limitado.'}

--- Accediendo a campos específicos ---
Sentimiento: Positivo
Calificación: 5


### Ventajas de la salida estructurada

Con este enfoque, ahora podemos:
- Almacenar los datos en una base de datos directamente
- Realizar análisis estadísticos sobre las calificaciones
- Filtrar reseñas por sentimiento
- Generar reportes automáticos

**Reflexión**: ¿Cómo usarías este sistema para procesar 1000 reseñas y generar un dashboard de métricas?

---

## 6. Serialización de Prompts

En aplicaciones reales, queremos guardar nuestros prompts en archivos para:
- Versionar los prompts (como código)
- Compartirlos con el equipo
- Modificarlos sin cambiar el código
- Reutilizarlos en diferentes proyectos

LangChain permite serializar prompts en formato JSON o YAML.

In [12]:
# Guardamos nuestro prompt en un archivo JSON
import json

# Convertimos el prompt a formato serializable
prompt_dict = {
    "template": template_resena,
    "input_variables": ["resena", "nombre_restaurante"],
    "template_format": "f-string"
}

# Guardamos en un archivo
with open("prompt_analisis_resenas.json", "w", encoding="utf-8") as f:
    json.dump(prompt_dict, f, indent=2, ensure_ascii=False)

print("Prompt guardado en: prompt_analisis_resenas.json")

# Leemos el prompt desde el archivo
with open("prompt_analisis_resenas.json", "r", encoding="utf-8") as f:
    prompt_cargado = json.load(f)

# Recreamos el PromptTemplate desde el archivo
prompt_desde_archivo = PromptTemplate(
    template=prompt_cargado["template"],
    input_variables=prompt_cargado["input_variables"]
)

# Verificamos que funciona correctamente
print("\nPrompt cargado desde archivo:")
print(prompt_desde_archivo.format(
    resena="Prueba de carga",
    nombre_restaurante="Restaurante Test"
))

Prompt guardado en: prompt_analisis_resenas.json

Prompt cargado desde archivo:

Eres un analista de reseñas de restaurantes. Tu trabajo es analizar la siguiente reseña y proporcionar:
1. Sentimiento general (Positivo, Negativo o Neutral)
2. Aspectos destacados (comida, servicio, ambiente, precio)
3. Calificación estimada del 1 al 5

Reseña del cliente:
Prueba de carga

Restaurante: Restaurante Test

Proporciona un análisis claro y conciso.



### Beneficios prácticos

Imagina que tu equipo de marketing quiere ajustar el tono del análisis. En lugar de modificar el código Python, simplemente editan el archivo JSON. El código de la aplicación permanece intacto.

Esto es especialmente útil cuando trabajas con prompt engineers o personas no técnicas que necesitan modificar las instrucciones del modelo.

---

## 7. Document Loaders: Trabajando con Documentos Externos

Hasta ahora trabajamos con texto que escribimos directamente. Pero las aplicaciones reales necesitan procesar documentos: PDFs, páginas web, archivos de texto, etc.

### Caso práctico: Análisis del menú de un restaurante

Vamos a crear un documento PDF de ejemplo que contenga el menú de un restaurante, y luego lo procesaremos con LangChain.

In [13]:
# Primero, creamos un documento de texto de ejemplo (simulando un menú)
contenido_menu = """
RESTAURANTE LA BUENA MESA
Menú Principal - 2024

ENTRADAS
- Ensalada César: Lechuga romana, parmesano, crutones, aderezo César. $45
- Bruschetta: Pan tostado con tomate fresco, albahaca y aceite de oliva. $38
- Sopa del día: Consultar con el mesero. $42

PLATOS PRINCIPALES
- Pasta Alfredo: Fettuccine con salsa cremosa de queso parmesano. $125
- Pechuga a la plancha: Con vegetales salteados y puré de papa. $145
- Salmón al horno: Con salsa de limón y ensalada fresca. $180
- Risotto de hongos: Arroz arborio con hongos silvestres y trufa. $135

POSTRES
- Tiramisú: Tradicional postre italiano con café y mascarpone. $55
- Cheesecake: Con salsa de frutos rojos. $60
- Helado artesanal: Sabores: vainilla, chocolate, fresa. $45

BEBIDAS
- Agua mineral: $25
- Refrescos: $30
- Jugos naturales: Naranja, zanahoria, verde. $40
- Café americano: $35
- Café latte: $45

INFORMACIÓN IMPORTANTE
- Todos los precios incluyen IVA
- Aceptamos todas las tarjetas de crédito
- Servicio de entrega a domicilio disponible
- Horario: Lunes a Domingo, 12:00 - 23:00 hrs
"""

# Guardamos el contenido en un archivo de texto
with open("menu_restaurante.txt", "w", encoding="utf-8") as f:
    f.write(contenido_menu)

print("Documento de menú creado: menu_restaurante.txt")

Documento de menú creado: menu_restaurante.txt


### Cargando el documento con LangChain

Ahora usamos un Document Loader para cargar el archivo. LangChain convierte el documento en objetos `Document` que podemos procesar fácilmente.

In [16]:
# Importamos el loader de archivos de texto
from langchain_community.document_loaders import TextLoader

# Cargamos el documento
loader = TextLoader("menu_restaurante.txt", encoding="utf-8")
documentos = loader.load()

# Exploramos el documento cargado
print(f"Número de documentos cargados: {len(documentos)}")
print(f"\nTipo de objeto: {type(documentos[0])}")
print(f"\nContenido (primeros 300 caracteres):")
print(documentos[0].page_content[:300])
print(f"\nMetadata del documento:")
print(documentos[0].metadata)

Número de documentos cargados: 1

Tipo de objeto: <class 'langchain_core.documents.base.Document'>

Contenido (primeros 300 caracteres):

RESTAURANTE LA BUENA MESA
Menú Principal - 2024

ENTRADAS
- Ensalada César: Lechuga romana, parmesano, crutones, aderezo César. $45
- Bruschetta: Pan tostado con tomate fresco, albahaca y aceite de oliva. $38
- Sopa del día: Consultar con el mesero. $42

PLATOS PRINCIPALES
- Pasta Alfredo: Fettucci

Metadata del documento:
{'source': 'menu_restaurante.txt'}


### Tipos de Document Loaders

LangChain soporta muchos tipos de documentos:
- **TextLoader**: Archivos de texto plano
- **PyPDFLoader**: Documentos PDF
- **CSVLoader**: Archivos CSV
- **UnstructuredHTMLLoader**: Páginas HTML
- **DirectoryLoader**: Carpetas completas

Para archivos PDF, usarías:
```python
# Para archivos PDF, usarías:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("documento.pdf")
pages = loader.load()
```

---

## 8. Resumen Automático de Documentos

Una vez que tenemos el documento cargado, podemos usar el LLM para extraer información, resumir, responder preguntas, etc. Vamos a crear un sistema que genere resúmenes automáticos del menú.

In [17]:
# Importamos las herramientas para resumir documentos
from langchain.chains.summarize import load_summarize_chain

# Estrategia 1: Resumen directo (para documentos pequeños)
# El documento completo se envía al modelo en una sola llamada

template_resumen = """
Escribe un resumen ejecutivo del siguiente menú de restaurante.
Incluye:
1. Tipo de cocina
2. Rango de precios
3. Especialidades destacadas
4. Información práctica importante

Menú:
{text}

Resumen ejecutivo:
"""

prompt_resumen = PromptTemplate(template=template_resumen, input_variables=["text"])

# Creamos la cadena de resumen
cadena_resumen = load_summarize_chain(
    llm,
    chain_type="stuff",  # "stuff" = meter todo el documento en una sola llamada
    prompt=prompt_resumen,
    verbose=False
)

# Generamos el resumen
resumen = cadena_resumen.invoke({"input_documents": documentos})

print("Resumen del menú:")
print(resumen["output_text"])

Resumen del menú:
**RESTAURANTE LA BUENA MESA - Resumen Ejecutivo del Menú 2024**

El menú de La Buena Mesa ofrece una propuesta culinaria atractiva, combinando la **cocina italiana con opciones internacionales** para satisfacer diversos gustos.

**1. Tipo de cocina:**
Principalmente **italiana**, con platos clásicos de pasta y risotto, complementada con opciones **internacionales y mediterráneas** (carnes, pescados, ensaladas).

**2. Rango de precios:**
Se posiciona en un segmento **medio-alto**.
*   **Entradas:** $38 - $45
*   **Platos Principales:** $125 - $180
*   **Postres:** $45 - $60
*   **Bebidas:** $25 - $45

**3. Especialidades destacadas:**
El menú sobresale por sus clásicos italianos como la **Pasta Alfredo** y el sofisticado **Risotto de hongos con trufa**. También ofrece opciones internacionales saludables como el **Salmón al horno con salsa de limón** y la **Pechuga a la plancha con vegetales**. Para el postre, el tradicional **Tiramisú** es una elección destacada.

**4.

### Estrategias de resumen

LangChain ofrece varias estrategias para resumir documentos:

1. **stuff**: Envía todo el documento en una sola llamada. Rápido pero limitado por el contexto del modelo.

2. **map_reduce**: Resume cada sección por separado, luego combina los resúmenes. Ideal para documentos largos.

3. **refine**: Resume iterativamente, refinando el resumen con cada sección. Más preciso pero más lento.

Para documentos grandes, usarías:
```python
cadena_resumen = load_summarize_chain(llm, chain_type="map_reduce")
```

---

## 9. Creación de Agentes con Herramientas

Hasta ahora, hemos usado LLMs de forma directa. Los agentes son más sofisticados: pueden razonar, tomar decisiones y usar herramientas para completar tareas complejas.

### ¿Qué es un agente?

Un agente es un sistema que:
1. Recibe una tarea
2. Decide qué herramientas usar
3. Ejecuta las herramientas necesarias
4. Interpreta los resultados
5. Repite hasta completar la tarea

### Caso práctico: Asistente de consultas de menú

Vamos a crear un agente que pueda:
- Buscar información específica del menú
- Calcular precios totales
- Responder preguntas sobre disponibilidad

In [18]:
# Primero, definimos herramientas personalizadas para nuestro agente
from langchain.agents import tool

@tool
def buscar_plato_en_menu(nombre_plato: str) -> str:
    """
    Busca un plato en el menú y devuelve su descripción y precio.

    Args:
        nombre_plato: El nombre del plato a buscar (ej: "Pasta Alfredo")

    Returns:
        str: Descripción y precio del plato, o mensaje de no encontrado
    """
    # Cargamos el menú
    with open("menu_restaurante.txt", "r", encoding="utf-8") as f:
        menu = f.read()

    # Buscamos el plato (búsqueda simple)
    lineas = menu.split("\n")
    for linea in lineas:
        if nombre_plato.lower() in linea.lower():
            return linea.strip()

    return f"No se encontró el plato '{nombre_plato}' en el menú"

@tool
def calcular_total(platos: str) -> str:
    """
    Calcula el total de una orden sumando los precios de los platos.

    Args:
        platos: Lista de platos separados por comas (ej: "Pasta Alfredo, Tiramisú")

    Returns:
        str: Total calculado en pesos
    """
    # Cargamos el menú
    with open("menu_restaurante.txt", "r", encoding="utf-8") as f:
        menu = f.read()

    platos_lista = [p.strip() for p in platos.split(",")]
    total = 0
    detalles = []

    for plato in platos_lista:
        # Buscamos el precio del plato
        for linea in menu.split("\n"):
            if plato.lower() in linea.lower() and "$" in linea:
                # Extraemos el precio
                try:
                    precio = int(linea.split("$")[1].strip().split()[0])
                    total += precio
                    detalles.append(f"{plato}: ${precio}")
                except:
                    continue

    if not detalles:
        return "No se pudo calcular el total. Verifica los nombres de los platos."

    resultado = "\n".join(detalles)
    resultado += f"\n\nTotal: ${total}"
    return resultado

# Verificamos que las herramientas funcionan
print("Prueba de herramienta 1:")
print(buscar_plato_en_menu.invoke({"nombre_plato": "Salmón"}))
print("\nPrueba de herramienta 2:")
print(calcular_total.invoke({"platos": "Pasta Alfredo, Tiramisú, Café latte"}))

Prueba de herramienta 1:
- Salmón al horno: Con salsa de limón y ensalada fresca. $180

Prueba de herramienta 2:
Pasta Alfredo: $125
Tiramisú: $55
Café latte: $45

Total: $225


### Creando el agente

Ahora configuramos el agente que usará estas herramientas de forma inteligente. El agente decidirá cuándo y cómo usar cada herramienta según la consulta del usuario.

In [19]:
# Importamos las clases necesarias para crear el agente
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

# Definimos las herramientas que el agente puede usar
herramientas = [buscar_plato_en_menu, calcular_total]

# Creamos el prompt del agente
# Este prompt le dice al agente su rol y cómo debe comportarse
prompt_agente = ChatPromptTemplate.from_messages([
    ("system", """Eres un asistente virtual de un restaurante.
    Tu trabajo es ayudar a los clientes con consultas sobre el menú.

    Puedes:
    - Buscar información de platos específicos
    - Calcular el total de una orden
    - Responder preguntas generales sobre el restaurante

    Sé amable y profesional. Si no estás seguro, usa las herramientas disponibles."""),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# Creamos el agente
agente = create_tool_calling_agent(llm, herramientas, prompt_agente)

# Creamos el ejecutor del agente
# El ejecutor es quien realmente corre el agente y maneja el ciclo de decisiones
ejecutor_agente = AgentExecutor(
    agent=agente,
    tools=herramientas,
    verbose=True,  # Para ver el proceso de razonamiento del agente
    max_iterations=5,  # Límite de iteraciones para evitar bucles infinitos
    handle_parsing_errors=True
)

print("Agente creado correctamente")

Agente creado correctamente


### Probando el agente

Ahora vamos a interactuar con nuestro agente. Observa cómo toma decisiones sobre qué herramientas usar.

In [20]:
# Prueba 1: Consulta simple sobre un plato
print("=" * 60)
print("CONSULTA 1: Información de un plato")
print("=" * 60)

respuesta1 = ejecutor_agente.invoke({
    "input": "¿Cuánto cuesta el Salmón al horno?"
})

print("\nRespuesta final:", respuesta1["output"])

CONSULTA 1: Información de un plato


> Entering new AgentExecutor chain...

Invoking: `buscar_plato_en_menu` with `{'nombre_plato': 'Salmón al horno'}`


- Salmón al horno: Con salsa de limón y ensalada fresca. $180El Salmón al horno cuesta $180.

> Finished chain.

Respuesta final: El Salmón al horno cuesta $180.


In [21]:
# Prueba 2: Cálculo de total (requiere usar herramienta calcular_total)
print("\n" + "=" * 60)
print("CONSULTA 2: Cálculo de total")
print("=" * 60)

respuesta2 = ejecutor_agente.invoke({
    "input": "Quiero ordenar una Pasta Alfredo, una Ensalada César y un Café latte. ¿Cuánto sería el total?"
})

print("\nRespuesta final:", respuesta2["output"])


CONSULTA 2: Cálculo de total


> Entering new AgentExecutor chain...

Invoking: `calcular_total` with `{'platos': 'Pasta Alfredo, Ensalada César, Café latte'}`


Pasta Alfredo: $125
Ensalada César: $45
Café latte: $45

Total: $215El total de su orden, que incluye una Pasta Alfredo, una Ensalada César y un Café latte, sería de $215.


> Finished chain.

Respuesta final: El total de su orden, que incluye una Pasta Alfredo, una Ensalada César y un Café latte, sería de $215.



In [22]:
# Prueba 3: Pregunta general (no requiere herramientas)
print("\n" + "=" * 60)
print("CONSULTA 3: Pregunta general")
print("=" * 60)

respuesta3 = ejecutor_agente.invoke({
    "input": "¿Cuál es el horario del restaurante?"
})

print("\nRespuesta final:", respuesta3["output"])


CONSULTA 3: Pregunta general


> Entering new AgentExecutor chain...
Lo siento, no tengo información sobre el horario del restaurante. Mi función es ayudarte con consultas sobre el menú y calcular totales de órdenes.

> Finished chain.

Respuesta final: Lo siento, no tengo información sobre el horario del restaurante. Mi función es ayudarte con consultas sobre el menú y calcular totales de órdenes.


### Análisis del comportamiento del agente

Observa el proceso de razonamiento del agente (si verbose=True):

1. **Pensamiento**: El agente analiza la pregunta
2. **Acción**: Decide qué herramienta usar
3. **Observación**: Recibe el resultado de la herramienta
4. **Pensamiento**: Decide si necesita más información o puede responder
5. **Respuesta final**: Genera la respuesta para el usuario

Este ciclo de razonamiento es lo que hace que los agentes sean tan poderosos: pueden resolver problemas complejos dividiéndolos en pasos más pequeños.

### Ejercicio de extensión

Intenta agregar una nueva herramienta al agente. Por ejemplo:

```python
@tool
def recomendar_plato(tipo: str) -> str:
    '''
    Recomienda un plato según el tipo solicitado.
    
    Args:
        tipo: Tipo de plato (entrada, principal, postre)
    
    Returns:
        str: Recomendación de plato
    '''
    # Tu implementación aquí
    pass
```

Luego, agrega la herramienta a la lista y vuelve a crear el agente.

---

## Conclusiones y Siguientes Pasos

### Lo que aprendimos

En este taller hemos cubierto los fundamentos de LangChain y agentes inteligentes:

1. **Configuración**: Cómo configurar el entorno y conectar con modelos de lenguaje
2. **PromptTemplates**: Diseñar prompts reutilizables y dinámicos
3. **Output Parsers**: Extraer datos estructurados de las respuestas del modelo
4. **Serialización**: Guardar y cargar configuraciones de prompts
5. **Document Loaders**: Procesar documentos externos
6. **Resumen automático**: Generar resúmenes de documentos grandes
7. **Agentes**: Crear sistemas que toman decisiones y usan herramientas

### Aplicaciones prácticas

Con estos conocimientos puedes construir:

- Sistemas de análisis de sentimientos para reseñas
- Chatbots inteligentes para atención al cliente
- Herramientas de resumen de documentos largos
- Asistentes virtuales con acceso a bases de datos
- Sistemas de recomendación personalizados
- Automatización de análisis de datos textuales

### Próximos pasos recomendados

1. **Memoria conversacional**: Agregar memoria a los agentes para mantener contexto entre conversaciones
2. **Bases de datos vectoriales**: Usar FAISS o Pinecone para búsquedas semánticas avanzadas
3. **Chains avanzadas**: Explorar chains más complejas como RetrievalQA
4. **Agentes especializados**: Crear agentes con capacidades específicas de dominio
5. **Integración con APIs**: Conectar agentes con servicios externos
6. **Evaluación y testing**: Implementar métricas para evaluar la calidad de las respuestas

### Recursos adicionales

- Documentación oficial de LangChain: https://python.langchain.com/
- Gemini API: https://ai.google.dev/
- Comunidad de LangChain: https://github.com/langchain-ai/langchain

---

**Felicitaciones por completar este taller.** Has dado el primer paso en el desarrollo de aplicaciones inteligentes con LLMs.